In [11]:
import whisper
from pprint import pprint
import pandas as pd

In [4]:
model = whisper.load_model("base", device="cpu")

In [5]:
result = model.transcribe("sample1.flac", word_timestamps=True, fp16=False)

In [ ]:
words = []
for seg in result["segments"]:
    for w in seg["words"]:
        words.append({
            "word": w["word"].strip(),
            "start": float(w["start"]),
            "end": float(w["end"]),
            "duration": float(w["end"] - w["start"]),
            "confidence": float(w["probability"])
        })

df_words = pd.DataFrame(words)
df_words.sort_values("confidence").head(5)


,word,start,end,duration,confidence
0,going,0.00,0.36,0.36,0.610064
17,fortnight.,5.88,6.40,0.52,0.659190
12,schoolrooms,4.36,4.92,0.56,0.707391
11,drafty,3.88,4.36,0.48,0.744157
38,us,12.12,12.28,0.16,0.772066


In [ ]:
segments = []
for seg in result["segments"]:
    segments.append({
        "text": seg["text"].strip(),
        "start": float(seg["start"]),
        "end": float(seg["end"]),
        "duration": float(float(seg["end"]) - float(seg["start"])),
        "avg_word_confidence": sum([float(w["probability"]) for w in seg["words"]]) / (len(seg["words"]) if len(seg["words"]) > 0 else 0.0)
    })

df_segments = pd.DataFrame(segments)
with pd.option_context('display.max_colwidth', None):
    print(df_segments.head()) 

                                                                                                text  \
0    going along slushy country roads and speaking to damp audiences in drafty schoolrooms day after   
1  day for fortnight. He'll have to put in an appearance at some place of worship on Sunday morning,   
2                                                      and he can come to us immediately afterwards.   

   start    end  duration  avg_word_confidence  
0   0.00   5.46      5.46             0.896980  
1   5.46  10.98      5.52             0.950516  
2  11.46  13.34      1.88             0.963631  
